In [ ]:
import torch
import torchvision as tv
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
%%bash

rm -rf .data
mkdir -p .data/faces
unzip -q att-database-of-faces.zip -d .data/faces-training
cp -a .data/faces-training .data/faces-test

rm .data/faces-training/*/{1,2,3,4,5}.pgm
rm .data/faces-test/*/{6,7,8,9,10}.pgm


In [ ]:
torch.manual_seed(1)

t = tv.transforms.Compose([
    tv.transforms.Grayscale(),
    tv.transforms.ToTensor()
])

# Load AT&T database of faces.
dataset = tv.datasets.ImageFolder(root=".data/faces-training", transform=t)


In [ ]:
target_person = 37

all_images_of_target = [img for img, label in dataset if label == target_person]

_, ax = plt.subplots(1, len(all_images_of_target), figsize=(20, 5))
    
for p, img in zip(ax, all_images_of_target):
    p.imshow(img.squeeze(), cmap="gray")
    p.axis("off")

plt.show()

In [ ]:
nc = 40
nf = 112 * 92

model = torch.nn.Linear(nf, nc)

opt = torch.optim.SGD(model.parameters(), lr=0.1)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=True)

n_epochs = 20
cost = []

for i in range(n_epochs):
    l = 0
    n = 0
    for img, labels in loader:
        img = img.view(-1, nf)  # from [nbatches, 1, 112, 92] to [nbatches, 10304]
        output = model(img)
        opt.zero_grad()
        loss = criterion(output, labels)
        loss.backward()
        
        #############################################################
        # Enable the following lines to simulate differential privacy
        #############################################################
        #rnd = torch.distributions.normal.Normal(0.0, 1.0)
        #for p in model.parameters():
        #    p.grad += rnd.sample(torch.Size(p.grad.shape)) * 0.07
        #############################################################

        opt.step()
        l += loss.item()
        n += 1
    print(i, l/n)
    cost.append(l/n)
        
plt.plot(cost)
plt.show()

In [ ]:
dataset = tv.datasets.ImageFolder(root=".data/faces-test", transform=t)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=200, shuffle=True)

with torch.no_grad():
    img, labels = iter(test_loader).next()
    r = model(img.view(-1, nf))
    p = r.argmax(dim=1)
    print("images:", len(labels))
    print("accuracy:", (labels == p).sum().item() / len(labels))


In [ ]:
import torch.nn.functional as F

x = torch.zeros(nf, requires_grad=True)
o = torch.optim.SGD([x], lr=0.1)

for i in range(1000):
    scores = F.softmax(model(x.view(1, nf)), dim=1).squeeze()
    e = torch.tensor([1.0]) - scores[target_person] # error for the target label
    o.zero_grad()
    e.backward()
    o.step()
    
x

In [ ]:
r = F.softmax(model(x), dim=0)
print("score of target person:", r[target_person].item())
print("scores:")
r

In [ ]:
img = x.view(112, 92).detach()

plt.imshow(img, cmap="gray")
plt.show()